In [250]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np

# Auth
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('gspread_creds.json', scope)
client = gspread.authorize(creds)


### TFR-500 list

In [251]:
tfr500 = pd.read_excel('AGs in TFR-500.xlsx')
tfr500 = tfr500.rename(columns={'Firm name': 'Master name'})

### Handbuch 1932

In [252]:
# Open the sheet
worksheet = client.open("Handbuch 1932").worksheet("validated")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
hb32 = pd.DataFrame(records)

In [253]:
bool_cols = ['Grossaktionär is a US comp', 
             'Director attached to a US company', 
             'US location linked to a director', 
             'US-sounding names', 
             'Bond issuance in the US', 
             'Other indicators are strong']
hb32[bool_cols] = hb32[bool_cols].apply(lambda x: x == "TRUE")

In [254]:
hb32['Included in the list'] = hb32[['Grossaktionär is a US comp', 
                                 'Director attached to a US company', 
                                 'US location linked to a director', 
                                 'US-sounding names',
                                 'Other indicators are strong']].any(axis=1)

hb32

,ab_firm_id,firmname,band,firmname_page,page_num,corrected firm name,Master name,US parent,Master US firm name,Grossaktionär is a US comp,Director attached to a US company,US location linked to a director,US-sounding names,Bond issuance in the US,Other indicators,Other indicators are strong,Included in the list
0,30728,„Miag“ Mühlenbau u. Industrie Aktiengesellschaft,3,543,546,„Miag“ Mühlenbau u. Industrie Aktiengesellschaft,„Miag“ Mühlenbau u. Industrie Aktiengesellschaft,,,False,False,True,False,True,,False,True
1,20200,Aetien-Bauverein „Passage“,2,326,326,Actien-Bauverein „Passage“,,,,False,False,False,False,False,,False,False
2,11541,Actien-Bierbrauerei,1,1249,1250,Actien-Bierbrauerei,Actien-Bierbrauerei,,,False,False,True,False,False,,False,True
3,41429,Adam Opel Aktiengesellschaft,4,765,766,Adam Opel Aktiengesellschaft,Adam Opel Aktiengesellschaft,General Motors Corporation. New York.,General Motors Corporation (Overseas Operations),True,False,False,False,False,,False,True
4,30162,Akt.-Ges. für Erwerb und Verwaltung von Grundb...,3,271,271,Akt.-Ges. für Erwerb und Verwaltung von Grundb...,Akt.-Ges. für Erwerb und Verwaltung von Grundb...,,,False,False,True,False,False,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,,,,,,,,,,False,False,False,False,False,,False,False
967,,,,,,,,,,False,False,False,False,False,,False,False
968,,,,,,,,,,False,False,False,False,False,,False,False
969,,,,,,,,,,False,False,False,False,False,,False,False


In [255]:
hb32_validated = hb32[hb32['Included in the list']]

#  Define priority order
priority_cols = ['Grossaktionär is a US comp', 'Director attached to a US company', 'US location linked to a director', 'US-sounding names', 'Other indicators are strong']

# Use np.select to assign indicator based on order
conditions = [hb32_validated[col] for col in priority_cols]
choices = priority_cols

hb32_validated["indicator"] = np.select(conditions, choices, default=None)

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/2945425234.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hb32_validated["indicator"] = np.select(conditions, choices, default=None)


In [256]:
hb32_validated_list = hb32_validated['Master name']

In [257]:
tfr500_hb32 = tfr500.merge(hb32_validated_list, on='Master name', indicator=True, how='outer')
tfr500_hb32['TFR-500'] = tfr500_hb32['_merge'].isin(['left_only', 'both'])
tfr500_hb32['HB 32 validated'] = tfr500_hb32['_merge'].isin(['right_only', 'both'])
tfr500_hb32 = tfr500_hb32.drop(columns='_merge')

In [258]:
tfr500_hb32

,Master name,US parent,TFR-500,HB 32 validated
0,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False
1,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False
2,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False
3,Actien-Bierbrauerei,NaN,False,True
4,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True
...,...,...,...,...
156,Verkehrs & Handels Aktiengesellschaft,United Continental Corporation,True,False
157,Warsteiner und Herzoglich Schleswig-Holsteinis...,NaN,False,True
158,Wrigley Aktiengesellschaft,NaN,False,True
159,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False


### Handbuch 1934

In [259]:
# Open the sheet
worksheet = client.open("Handbuch 1934").worksheet("validated")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
hb34 = pd.DataFrame(records)

In [260]:
hb34

,firm_id,firmname,page,section_title,text,US Location Mention,US-Resembling Company Name,Indicator of US Investment?,Reasoning,band,...,Type of US ownership,Grossaktionär is a US comp,Director attached to a US company,US location linked to a director,US-sounding names,Bond issuance in the US,US parent explicitly mentioned,Other indicators,Other indicators are strong,us-sounding name only
0,42,Bremen-Amerika Bank Aktiengesellschaft.,202,Aufsichtsrat,Aufsichtsrat: Vors.: Gen.-Konsul Dr. h. c. Lud...,No,Yes (Amerika in name),Yes,The company name 'Bremen-Amerika Bank' explici...,91,...,other (not likely),FALSE,FALSE,FALSE,FALSE,FALSE,,,FALSE,
1,44,Bremer Liquidationskasse von 1924 Akt.-Ges.,203,Vorstand,"Vorstand: Pani Braun, Hans Heine. Aufsiclitsra...",No,Yes (Deutsch-Amerikan. in name),Yes,A board member is associated with a firm expli...,91,...,other (not likely),FALSE,FALSE,FALSE,FALSE,FALSE,,,FALSE,
2,106,Süddeutsche Festwertbank Aktiengesellschaft.,263,Großaktionär,Großaktionäre: Das A.-K. dei Ges. ist im Be­ s...,Yes (New York),No,Yes,Explicit mention of 'New York' in the context ...,91,...,other (not likely),FALSE,FALSE,FALSE,FALSE,FALSE,,,FALSE,
3,268,für Bergbau und Hüttenbetrieh.,331,Aufsichtsrat,"Aufsichtsrat: Vors.: Dr. Paul von Schwabach, ...",Yes (New York),Yes (W. A. Harriman & Co.),Yes,Explicit mention of 'New York' associated with...,91,...,Director with US location,FALSE,TRUE,TRUE,FALSE,FALSE,,,FALSE,
4,289,"Gutehoffnungshütte, Aktienverein für Bergbau u...",361,Großaktionär,Großaktionäre: Familie Haniel; eine von den T...,Yes ($),Weak/No (Company Name itself) but Yes (Context),Yes,Explicit mention of US dollars ($) in the bond...,91,...,bond issuance in the US,FALSE,FALSE,FALSE,FALSE,TRUE,,,FALSE,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2581,Dresdner Gardinen- und Spitzen-Manufactur Acti...,1048,Aufsichtsrat,"Aufsichtsrat: Vors.: Fabrikbes. Otto Lisch, D...","Yes (Norwalk, Conn. U. S. A.)",Potentially (The Dresden Lace Work Ine.),Yes,"Explicit mention of Norwalk, Connecticut (U. S...",94,...,other (not likely),FALSE,FALSE,FALSE,FALSE,FALSE,,,FALSE,
120,2645,Hirsch-Bräu Aktiengesellschaft.,1082,Prokuristen,"Prokuristen: K. Mennicke, P. Cremer. Anfsichts...",Yes (New York),No,Yes,Explicit US location 'New York' in relation to...,94,...,Director with US location,FALSE,FALSE,TRUE,FALSE,FALSE,,,FALSE,
121,2807,"Ha-No-Mex, Handels-Aktiengesellschaft in Liqu.",1141,Aufsichtsrat,Aufsichtsrat: Vors.: Rechtsanw. Dr. Walter Ma...,"Yes (New York, Brookline)",No,Yes,Paragraph 10 indicates US investment due to th...,94,...,Director with US location,FALSE,FALSE,TRUE,FALSE,FALSE,,,FALSE,
122,2870,in Liqu.,1164,Aufsichtsrat,"Aufsiehtsrat: Rechtsanw. Dr. Karl Hirschland, ...",No,No,Yes,Mentions an 'amerik. Rechtsanw. Frederik Wirth...,94,...,Director with US location,FALSE,FALSE,FALSE,FALSE,FALSE,,Mentions an 'amerik. Rechtsanw. ' on the board,TRUE,


In [261]:
bool_cols = ['Grossaktionär is a US comp', 
             'Director attached to a US company', 
             'US location linked to a director', 
             'US-sounding names', 
             'Bond issuance in the US', 
             'Other indicators are strong']
hb34[bool_cols] = hb34[bool_cols].apply(lambda x: x == "TRUE")

In [262]:
hb34['Included in the list'] = hb34[['Grossaktionär is a US comp', 
                                 'Director attached to a US company', 
                                 'US location linked to a director', 
                                 'US-sounding names',
                                 'Other indicators are strong']].any(axis=1)

hb34

,firm_id,firmname,page,section_title,text,US Location Mention,US-Resembling Company Name,Indicator of US Investment?,Reasoning,band,...,Grossaktionär is a US comp,Director attached to a US company,US location linked to a director,US-sounding names,Bond issuance in the US,US parent explicitly mentioned,Other indicators,Other indicators are strong,us-sounding name only,Included in the list
0,42,Bremen-Amerika Bank Aktiengesellschaft.,202,Aufsichtsrat,Aufsichtsrat: Vors.: Gen.-Konsul Dr. h. c. Lud...,No,Yes (Amerika in name),Yes,The company name 'Bremen-Amerika Bank' explici...,91,...,False,False,False,False,False,,,False,,False
1,44,Bremer Liquidationskasse von 1924 Akt.-Ges.,203,Vorstand,"Vorstand: Pani Braun, Hans Heine. Aufsiclitsra...",No,Yes (Deutsch-Amerikan. in name),Yes,A board member is associated with a firm expli...,91,...,False,False,False,False,False,,,False,,False
2,106,Süddeutsche Festwertbank Aktiengesellschaft.,263,Großaktionär,Großaktionäre: Das A.-K. dei Ges. ist im Be­ s...,Yes (New York),No,Yes,Explicit mention of 'New York' in the context ...,91,...,False,False,False,False,False,,,False,,False
3,268,für Bergbau und Hüttenbetrieh.,331,Aufsichtsrat,"Aufsichtsrat: Vors.: Dr. Paul von Schwabach, ...",Yes (New York),Yes (W. A. Harriman & Co.),Yes,Explicit mention of 'New York' associated with...,91,...,False,True,True,False,False,,,False,,True
4,289,"Gutehoffnungshütte, Aktienverein für Bergbau u...",361,Großaktionär,Großaktionäre: Familie Haniel; eine von den T...,Yes ($),Weak/No (Company Name itself) but Yes (Context),Yes,Explicit mention of US dollars ($) in the bond...,91,...,False,False,False,False,True,,,False,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2581,Dresdner Gardinen- und Spitzen-Manufactur Acti...,1048,Aufsichtsrat,"Aufsichtsrat: Vors.: Fabrikbes. Otto Lisch, D...","Yes (Norwalk, Conn. U. S. A.)",Potentially (The Dresden Lace Work Ine.),Yes,"Explicit mention of Norwalk, Connecticut (U. S...",94,...,False,False,False,False,False,,,False,,False
120,2645,Hirsch-Bräu Aktiengesellschaft.,1082,Prokuristen,"Prokuristen: K. Mennicke, P. Cremer. Anfsichts...",Yes (New York),No,Yes,Explicit US location 'New York' in relation to...,94,...,False,False,True,False,False,,,False,,True
121,2807,"Ha-No-Mex, Handels-Aktiengesellschaft in Liqu.",1141,Aufsichtsrat,Aufsichtsrat: Vors.: Rechtsanw. Dr. Walter Ma...,"Yes (New York, Brookline)",No,Yes,Paragraph 10 indicates US investment due to th...,94,...,False,False,True,False,False,,,False,,True
122,2870,in Liqu.,1164,Aufsichtsrat,"Aufsiehtsrat: Rechtsanw. Dr. Karl Hirschland, ...",No,No,Yes,Mentions an 'amerik. Rechtsanw. Frederik Wirth...,94,...,False,False,False,False,False,,Mentions an 'amerik. Rechtsanw. ' on the board,True,,True


In [263]:
hb34_validated = hb34[hb34['Included in the list']]

#  Define priority order
priority_cols = ['Grossaktionär is a US comp', 'Director attached to a US company', 'US location linked to a director', 'US-sounding names', 'Other indicators are strong']

# Use np.select to assign indicator based on order
conditions = [hb34_validated[col] for col in priority_cols]
choices = priority_cols

hb34_validated["indicator"] = np.select(conditions, choices, default=None)

hb34_validated_list = hb34_validated['Master name']

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/3666419628.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hb34_validated["indicator"] = np.select(conditions, choices, default=None)


In [264]:
tfr500_hb32_hb34 = tfr500_hb32.merge(hb34_validated_list, on='Master name', indicator=True, how='outer')
tfr500_hb32_hb34['HB 34 validated'] = tfr500_hb32_hb34['_merge'].isin(['right_only', 'both'])
tfr500_hb32_hb34['TFR-500'] = tfr500_hb32_hb34['TFR-500'].fillna(False)
tfr500_hb32_hb34['HB 32 validated'] = tfr500_hb32_hb34['HB 32 validated'].fillna(False)
tfr500_hb32_hb34 = tfr500_hb32_hb34.drop(columns='_merge')
tfr500_hb32_hb34

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/4130509289.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hb32_hb34['TFR-500'] = tfr500_hb32_hb34['TFR-500'].fillna(False)
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/4130509289.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hb32_hb34['HB 32 validated'] = tfr500_hb32_hb34['HB 32 validated'].fillna(False)


,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated
0,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True
1,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False
2,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False
3,Actien-Bierbrauerei,NaN,False,True,True
4,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True,True
...,...,...,...,...,...
172,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True
173,Wrigley Aktiengesellschaft,NaN,False,True,True
174,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True
175,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False


In [265]:
tfr500_hb32_hb34[(tfr500_hb32_hb34['TFR-500']==False) & 
                 (
                     (tfr500_hb32_hb34['HB 32 validated']==True) | 
                     (tfr500_hb32_hb34['HB 34 validated']==True)
                )
                ]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated
3,Actien-Bierbrauerei,NaN,False,True,True
5,Akt.-Ges. für Erwerb und Verwaltung von Grundb...,NaN,False,True,True
7,Aktiengesellsehaft für Konzessionsunternehmen,NaN,False,True,True
8,"Alex. Zink, Filzfabrik, Akt.-Ges.",NaN,False,True,True
9,Allgemeine Automobil-Versicherungs-Aktiengesel...,NaN,False,True,False
...,...,...,...,...,...
171,Warsteiner und Herzoglich Schleswig-Holsteinis...,NaN,False,True,False
172,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True
173,Wrigley Aktiengesellschaft,NaN,False,True,True
174,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True


In [266]:
tfr500_hb32_hb34[(tfr500_hb32_hb34['TFR-500']==True) & 
                 (
                     (tfr500_hb32_hb34['HB 32 validated']==False) &
                     (tfr500_hb32_hb34['HB 34 validated']==False)
                )
                ]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated
1,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False
2,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False
6,Aktien-Malzfabrik Landsberg AG,Corn Products Refining company,True,False,False
12,Alpenminen A. G.,American-Austrian Magnesite Corporation,True,False,False
18,Bekleidungs AG,"New Jersey Industries, Inc.",True,False,False
21,Berliner Telephonie AG,International Telephone and Telegraph Corp.,True,False,False
23,Borvisk Kunstseiden AG,"Continental ""Borvisk"" Co.",True,False,False
24,Brankär Eisenwerke AG,The Yale & Towne Manufacturing Company,True,False,False
26,Busscher & Hoffman AG,Pan Foreign Corporation,True,False,False
27,C. H. Knorr AG,Corn Products Refining company,True,False,False


### Moody's 1932

In [267]:
# Open the sheet
worksheet = client.open("Moodys 1932").worksheet("list of AGs")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
moodys32 = pd.DataFrame(records)

In [268]:
moodys32 = moodys32.rename(columns={'German subsidiary': 'Master name'})
moodys32 = moodys32[['Master name']]

In [269]:
tfr500_hb32_hb34_moodys32 = tfr500_hb32_hb34.merge(moodys32, on='Master name', indicator=True, how='outer')
tfr500_hb32_hb34_moodys32['Moodys 32'] = tfr500_hb32_hb34_moodys32['_merge'].isin(['right_only', 'both'])
tfr500_hb32_hb34_moodys32['TFR-500'] = tfr500_hb32_hb34_moodys32['TFR-500'].fillna(False)
tfr500_hb32_hb34_moodys32['HB 32 validated'] = tfr500_hb32_hb34_moodys32['HB 32 validated'].fillna(False)
tfr500_hb32_hb34_moodys32['HB 34 validated'] = tfr500_hb32_hb34_moodys32['HB 34 validated'].fillna(False)
tfr500_hb32_hb34_moodys32 = tfr500_hb32_hb34_moodys32.drop(columns='_merge')
tfr500_hb32_hb34_moodys32

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/2621954440.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hb32_hb34_moodys32['TFR-500'] = tfr500_hb32_hb34_moodys32['TFR-500'].fillna(False)
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/2621954440.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hb32_hb34_moodys32['HB 32 validated'] = tfr500_hb32_hb34_moodys32['HB 32 validated'].fillna(False)
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/2621954440.py:5: FutureWarning: Downcasting

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32
0,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False
1,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False
2,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False
3,Actien-Bierbrauerei,NaN,False,True,True,False
4,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True,True,True
...,...,...,...,...,...,...
177,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False
178,Wrigley Aktiengesellschaft,NaN,False,True,True,False
179,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False
180,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False,False


### Moody's 1934

In [270]:
# Open the sheet
worksheet = client.open("Moodys 1934").worksheet("list of AGs")

# Get all records as list of dicts
records = worksheet.get_all_records()

# Convert to DataFrame
moodys34 = pd.DataFrame(records)

In [271]:
moodys34 = moodys34.rename(columns={'German subsidiary': 'Master name'})
moodys34 = moodys34[['Master name']]

In [272]:
tfr500_hb32_hb34_moodys32_moodys34 = tfr500_hb32_hb34_moodys32.merge(moodys34, on='Master name', indicator=True, how='outer')
tfr500_hb32_hb34_moodys32_moodys34['Moodys 34'] = tfr500_hb32_hb34_moodys32_moodys34['_merge'].isin(['right_only', 'both'])
tfr500_hb32_hb34_moodys32_moodys34['TFR-500'] = tfr500_hb32_hb34_moodys32_moodys34['TFR-500'].fillna(False)
tfr500_hb32_hb34_moodys32_moodys34['HB 32 validated'] = tfr500_hb32_hb34_moodys32_moodys34['HB 32 validated'].fillna(False)
tfr500_hb32_hb34_moodys32_moodys34['HB 34 validated'] = tfr500_hb32_hb34_moodys32_moodys34['HB 34 validated'].fillna(False)
tfr500_hb32_hb34_moodys32_moodys34['Moodys 32'] = tfr500_hb32_hb34_moodys32_moodys34['Moodys 32'].fillna(False)
tfr500_hb32_hb34_moodys32_moodys34 = tfr500_hb32_hb34_moodys32_moodys34.drop(columns='_merge')
tfr500_hb32_hb34_moodys32_moodys34

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34
0,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False
1,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False
2,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False
3,Actien-Bierbrauerei,NaN,False,True,True,False,False
4,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True,True,True,True
...,...,...,...,...,...,...,...
177,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False
178,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True
179,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False
180,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False,False,False


### Tenenbaum

In [273]:
tenenbaum = pd.read_excel('tenebaum_list_AGs.xlsx')

In [274]:
tenenbaum

,German subsidiary,US parent,Master name
0,Ford Motor A.G.,Ford Motor Company,Ford-Werke AG
1,Credit A.G. für Ford fahrzeuge,Ford Motor Company,Credit Aktiengesellschaft für Ford-Fahrzeuge.
2,Adam Opel A.G.,General Motors Corporation,Adam Opel Aktiengesellschaft
3,Opel Automobilversicherungs A.G.,General Motors Corporation,Opel Automobil Versicherungs AG
4,Continentale Gummiwerke A.G.,General Motors Corporation,Continentale Gummiwerke A.G.
5,Duco A. G.,E. I. du Pont de Nemours & Co.,Duco AG
6,Dynamit A. G.,E. I. du Pont de Nemours & Co.,Dynamit-Act.-Ges. vormals Alfred Nobel & Co.
7,"""Dom"" Finanz A. G. für chemische Unternehmenungen","Richard Hudnut, of New York","""Dom"" Finanz A. G. für chemische Unternehmenungen"
8,Goedecke & Co.,"Richard Hudnut, of New York",Gödecke & Co. Chemische Fabrik AG
9,Gustav Lohse A. G.,"Richard Hudnut, of New York",Gustav Lohse AG


In [275]:
tfr500_hbs_moodys_tenenbaum = tfr500_hb32_hb34_moodys32_moodys34.merge(tenenbaum[['Master name']], on='Master name', indicator=True, how='outer')
tfr500_hbs_moodys_tenenbaum['Tenenbaum'] = tfr500_hbs_moodys_tenenbaum['_merge'].isin(['right_only', 'both'])
tfr500_hbs_moodys_tenenbaum['TFR-500'] = tfr500_hbs_moodys_tenenbaum['TFR-500'].fillna(False)
tfr500_hbs_moodys_tenenbaum['HB 32 validated'] = tfr500_hbs_moodys_tenenbaum['HB 32 validated'].fillna(False)
tfr500_hbs_moodys_tenenbaum['HB 34 validated'] = tfr500_hbs_moodys_tenenbaum['HB 34 validated'].fillna(False)
tfr500_hbs_moodys_tenenbaum['Moodys 32'] = tfr500_hbs_moodys_tenenbaum['Moodys 32'].fillna(False)
tfr500_hbs_moodys_tenenbaum['Moodys 34'] = tfr500_hbs_moodys_tenenbaum['Moodys 34'].fillna(False)
tfr500_hbs_moodys_tenenbaum = tfr500_hbs_moodys_tenenbaum.drop(columns=['_merge'])
tfr500_hbs_moodys_tenenbaum

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/95407602.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hbs_moodys_tenenbaum['TFR-500'] = tfr500_hbs_moodys_tenenbaum['TFR-500'].fillna(False)
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/95407602.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hbs_moodys_tenenbaum['HB 32 validated'] = tfr500_hbs_moodys_tenenbaum['HB 32 validated'].fillna(False)
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/95407602.py:5: FutureWarning: Downcasti

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False
...,...,...,...,...,...,...,...,...
186,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False
187,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True,True
188,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False
189,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False,False,False,False


### Search for firms in Handbuch 1932

In [276]:
hb32_to_search = tfr500_hbs_moodys_tenenbaum[tfr500_hbs_moodys_tenenbaum['HB 32 validated']==False]
hb32_to_search.to_excel('hb32_to_search_auto.xlsx', index=False)

In [277]:
manual_search_hb32 = pd.read_excel('hb32_manual_searched.xlsx')
auto_manual_merged = hb32_to_search.merge(manual_search_hb32[['Master name', 
                                                              'ab_firm_id', 
                                                              'firmname',
                                                              'band', 
                                                              'firmname_page']], 
                                            on='Master name', 
                                            how='left')

In [278]:
# auto_manual_merged.to_excel('hb32_to_search_auto.xlsx', index=False)

In [279]:
auto_manual_merged

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,ab_firm_id,firmname,band,firmname_page
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,41670.0,„Debag“ Deutsche Betriebsstoff-Akt.-Ges.,4.0,1670.0
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,20224.0,„Dom“ Finanz-Aktiengesellschaft für chemische ...,2.0,224.0
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,21557.0,Hinds Aktiengesellschaft,2.0,1295.0
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,31682.0,Actiengesellschaft vorm. Seidel & Naumann,3.0,1682.0
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
91,W. A. Schölten Stärke- und Syrup-Fabriken Akti...,NaN,False,False,True,False,False,False,41566.0,W. A. Scholten Stärke- u. Syrupfabriken Akt.-Ges.,4.0,1566.0
92,Walsheim Brauerei Aktien-Gesellschaft vorm. Sc...,NaN,False,False,True,False,False,False,11422.0,Walsheim Brauerei-Aktiengesellschaft,1.0,1422.0
93,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False,20506.0,Werschen -Weissenfelser Braunkohlen-Aktien-Ges...,2.0,506.0
94,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False,31233.0,Zukunft Lebensversicherungsbank Akt.-Ges.,3.0,1233.0


In [280]:
auto_manual_merged['Present in HB 32'] = auto_manual_merged['ab_firm_id'].notnull()
tfr500_hbs_moodys_tenebaum_hb32search = tfr500_hbs_moodys_tenenbaum.merge(auto_manual_merged[['Master name', 'Present in HB 32']], on='Master name', how='left')
tfr500_hbs_moodys_tenebaum_hb32search['Present in HB 32'] = tfr500_hbs_moodys_tenebaum_hb32search['Present in HB 32'].fillna(True)
tfr500_hbs_moodys_tenebaum_hb32search

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/2894378363.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tfr500_hbs_moodys_tenebaum_hb32search['Present in HB 32'] = tfr500_hbs_moodys_tenebaum_hb32search['Present in HB 32'].fillna(True)


,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,True
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,True
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
186,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False,True
187,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True,True,True
188,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False,True
189,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False,False,False,False,True


### Search for firms in Handbuch 1934

In [281]:
hb34_to_search = tfr500_hbs_moodys_tenenbaum[tfr500_hbs_moodys_tenenbaum['HB 34 validated']==False]
hb34_to_search.to_excel('hb34_to_search_auto.xlsx', index=False)

In [282]:
manual_search_hb34 = pd.read_excel('hb34_manual_searched.xlsx')
hb34_auto_manual_merged = hb34_to_search.merge(manual_search_hb34[['Master name', 
                                                              'ab_firm_id', 
                                                              'firmname',
                                                              'band', 
                                                              'firmname_page']], 
                                            on='Master name', 
                                            how='left')

In [283]:
hb34_auto_manual_merged.to_excel('hb34_to_search_auto.xlsx', index=False)

In [284]:
hb34_auto_manual_merged

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,ab_firm_id,firmname,band,firmname_page
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,32460.0,„Debag“ Deutsche Betriebsstoff-Aktiengesellsch...,3.0,1373.0
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,32120.0,„Dom“ Finanz-Aktiengesellschaft für chem. Unte...,3.0,1242.0
2,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,20663.0,Aktiengesellschaft vorm. Seidel & Naumann.,2.0,600.0
3,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,30639.0,Aktiengesellschaft für Anlagewerte.,3.0,619.0
4,Aktien-Malzfabrik Landsberg AG,Corn Products Refining company,True,False,False,False,False,False,40180.0,Aktien-Malzfabrik Landsberg.,4.0,231.0
...,...,...,...,...,...,...,...,...,...,...,...,...
102,Vereinigte Bayerische Telephonwerke AG,International Telephone and Telegraph Corp.,True,False,False,False,False,True,21912.0,Vereinigte Bayerische Telephonwerke Aktiengese...,2.0,1549.0
103,Vereinigte Berliner Kohlenhandel AG,"Hugo Stinnes Industries, Inc.",True,False,False,False,False,False,32588.0,Vereinigte Berliner Kohlenhändler-Akt.-Ges.,3.0,1417.0
104,Verkehrs & Handels Aktiengesellschaft,United Continental Corporation,True,False,False,False,False,False,10272.0,Verkehrs- und Handels-Aktiengesellschaft.,1.0,334.0
105,Warsteiner und Herzoglich Schleswig-Holsteinis...,NaN,False,True,False,False,False,False,32312.0,Warsteiner und Herzoglich Schleswig-Holsteinis...,3.0,1313.0


In [285]:
hb34_auto_manual_merged['Present in HB 34'] = hb34_auto_manual_merged['ab_firm_id'].notnull()
master_hbs_searched = tfr500_hbs_moodys_tenebaum_hb32search.merge(hb34_auto_manual_merged[['Master name', 'Present in HB 34']], on='Master name', how='left')
master_hbs_searched['Present in HB 34'] = master_hbs_searched['Present in HB 34'].fillna(True)
master_hbs_searched

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/833759878.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  master_hbs_searched['Present in HB 34'] = master_hbs_searched['Present in HB 34'].fillna(True)


,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,True,True
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,True,True
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
186,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False,True,True
187,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
188,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False,True,True
189,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False,False,False,False,True,True


In [286]:
master_hbs_searched.to_excel('master_hbs_searched.xlsx', index=False)

#### TFR-500 firms not present in HB 32 nor 34

In [287]:
tfr500firms_not_in_hb = master_hbs_searched[(~master_hbs_searched['Present in HB 32']) & (~master_hbs_searched['Present in HB 34'])]
tfr500firms_not_in_hb

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
15,Alpenminen A. G.,American-Austrian Magnesite Corporation,True,False,False,False,False,False,False,False
22,Belton A.G.,NaN,False,False,False,False,False,True,False,False
25,Berliner Telephonie AG,International Telephone and Telegraph Corp.,True,False,False,False,False,True,False,False
28,Brankär Eisenwerke AG,The Yale & Towne Manufacturing Company,True,False,False,True,True,False,False,False
107,Intava Arbeitsgemeinschaft der Deutsch-Amerika...,"Socony-Vacuum Oil Company, Inc.; Standard Oil ...",True,False,False,False,False,False,False,False
109,Kaliwerke Sollstedt Gewerkschaft,NaN,False,False,False,False,False,True,False,False
117,Köhlmann Werke AG,Corn Products Refining company,True,False,False,False,False,False,False,False
143,OTA Schlesische Schuhwerke Ottmuth AG,Westhold Corporation,True,False,False,False,False,False,False,False
150,Paul Dorff AG,"Hugo Stinnes Industries, Inc.",True,False,False,False,False,False,False,False
151,Petrolifer Rohoeleompagnie AG,"Socony-Vacuum Oil Company, Inc.",True,False,False,False,False,False,False,False


In [288]:
tfr500firms_not_in_hb[['Master name', 'US parent']].to_excel('TFR firms in neither HB.xlsx', index=False)

#### TFR-500 firms present in HB 32 or 34 but not identified

In [289]:
master_hbs_searched[((master_hbs_searched['Present in HB 32'])| (master_hbs_searched['Present in HB 34'])) & 
                    ((master_hbs_searched['HB 32 validated']) | (master_hbs_searched['HB 34 validated']))]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,True,True
5,Actien-Bierbrauerei,NaN,False,True,True,False,False,False,True,True
6,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True,True,True,True,True,True,True
7,Akt.-Ges. für Erwerb und Verwaltung von Grundb...,NaN,False,True,True,False,False,False,True,True
10,Aktiengesellsehaft für Konzessionsunternehmen,NaN,False,True,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...
185,Warsteiner und Herzoglich Schleswig-Holsteinis...,NaN,False,True,False,False,False,False,True,True
186,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False,True,True
187,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
188,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False,True,True


In [290]:
master_hbs_searched[((master_hbs_searched['Present in HB 32'])| (master_hbs_searched['Present in HB 34'])) & 
                    ((~master_hbs_searched['HB 32 validated']) & (~master_hbs_searched['HB 34 validated']))]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,True,True
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,False,True
8,Aktien-Malzfabrik Landsberg AG,Corn Products Refining company,True,False,False,False,False,False,True,True
9,Aktiengesellschaft für Geschäftshausbau,NaN,False,False,False,True,False,False,True,True
21,Bekleidungs AG,"New Jersey Industries, Inc.",True,False,False,False,False,False,False,True
27,Borvisk Kunstseiden AG,"Continental ""Borvisk"" Co.",True,False,False,False,False,False,True,True
30,Busscher & Hoffman AG,Pan Foreign Corporation,True,False,False,False,False,False,True,True
31,C. H. Knorr AG,Corn Products Refining company,True,False,False,False,False,False,True,True


In [291]:
trf500firms_not_ided = master_hbs_searched[((master_hbs_searched['Present in HB 32'])| (master_hbs_searched['Present in HB 34'])) & 
                    ((~master_hbs_searched['HB 32 validated']) & (~master_hbs_searched['HB 34 validated']))]
trf500firms_not_ided.to_excel('trf500firms_not_ided_to_search.xlsx', index=False)

In [292]:
print(master_hbs_searched[(master_hbs_searched['TFR-500']) & (master_hbs_searched['Present in HB 32'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['TFR-500']) & (master_hbs_searched['Present in HB 34'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['TFR-500']) & (master_hbs_searched['Present in HB 32']) & (master_hbs_searched['Present in HB 34'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['TFR-500']) & 
                          ((master_hbs_searched['Present in HB 32']) | (master_hbs_searched['Present in HB 34']))].shape[0])

82
79
75
86


In [294]:
master_hbs_searched[(master_hbs_searched['Tenenbaum'])]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
6,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True,True,True,True,True,True,True
13,Allgemeine Elektricitäts-Gesellschaft,NaN,False,True,True,True,False,True,True,True
14,Allgemeine Transportmittel Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
22,Belton A.G.,NaN,False,False,False,False,False,True,False,False
25,Berliner Telephonie AG,International Telephone and Telegraph Corp.,True,False,False,False,False,True,False,False
32,C. Lorenz AG,International Standard Electric Corp.; Interna...,True,True,True,False,False,True,True,True
37,Continentale Gummiwerke A.G.,NaN,False,False,False,False,False,True,True,False
39,Credit Aktiengesellschaft für Ford-Fahrzeuge.,NaN,False,True,True,False,False,True,True,True


In [296]:
# How many Tenebaum firms are validated in HB 32 and 34?
# How many Tenebaum can be found in HB and 34?
# How many Tenebaum firms are in TFR-500?
# How many firms in TFR-500 are in Tenebaum?

# How many Tenebaum firms are validated in HB 32 and 34?
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['HB 32 validated'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['HB 34 validated'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['Moodys 32'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['Moodys 34'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['HB 32 validated']) & (master_hbs_searched['HB 34 validated'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & 
                          ((master_hbs_searched['HB 32 validated']) | (master_hbs_searched['HB 34 validated']))].shape[0])

# How many Tenebaum can be found in HB 32 and 34?
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['Present in HB 32'])].shape[0])
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['Present in HB 34'])].shape[0])

# How many Tenebaum firms are in TFR-500?
print(master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (master_hbs_searched['TFR-500'])].shape[0])


26
27
13
14
22
31
45
43
31


In [297]:
master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (~master_hbs_searched['HB 32 validated'])]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
22,Belton A.G.,NaN,False,False,False,False,False,True,False,False
25,Berliner Telephonie AG,International Telephone and Telegraph Corp.,True,False,False,False,False,True,False,False
37,Continentale Gummiwerke A.G.,NaN,False,False,False,False,False,True,True,False
47,Deutsche Burroughs Rechenmaschinen AG,Burrough Adding Machine Company,True,False,True,False,False,True,True,True
49,Deutsche Gasolin AG,Standard Oil Company (New Jersey),True,False,False,False,False,True,True,True
51,Deutsche Johnson A. G.,NaN,False,False,False,False,False,True,True,False
53,Deutsche Libbey-Owens Gesellschaft für machine...,NaN,False,False,False,True,True,True,True,True
56,Deutsche Tafelglas Aktiengesellschaft (Detag).,NaN,False,False,True,False,False,True,True,True


In [298]:
master_hbs_searched[(master_hbs_searched['Tenenbaum']) & (~master_hbs_searched['TFR-500'])]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
13,Allgemeine Elektricitäts-Gesellschaft,NaN,False,True,True,True,False,True,True,True
14,Allgemeine Transportmittel Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
22,Belton A.G.,NaN,False,False,False,False,False,True,False,False
37,Continentale Gummiwerke A.G.,NaN,False,False,False,False,False,True,True,False
39,Credit Aktiengesellschaft für Ford-Fahrzeuge.,NaN,False,True,True,False,False,True,True,True
51,Deutsche Johnson A. G.,NaN,False,False,False,False,False,True,True,False
53,Deutsche Libbey-Owens Gesellschaft für machine...,NaN,False,False,False,True,True,True,True,True
56,Deutsche Tafelglas Aktiengesellschaft (Detag).,NaN,False,False,True,False,False,True,True,True


In [300]:
master_hbs_searched[(~master_hbs_searched['Tenenbaum']) & (master_hbs_searched['TFR-500'])]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,True,True
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,True,True
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,False,True
8,Aktien-Malzfabrik Landsberg AG,Corn Products Refining company,True,False,False,False,False,False,True,True
15,Alpenminen A. G.,American-Austrian Magnesite Corporation,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
170,Südwest AG,"Hugo Stinnes Industries, Inc.",True,False,False,False,False,False,False,False
174,"Torpedo-Fahrräder u. Schreibmaschinen, Weilwer...",Remington Rand Inc.,True,True,False,False,False,False,True,True
178,Vereinigte Berliner Kohlenhandel AG,"Hugo Stinnes Industries, Inc.",True,False,False,False,False,False,True,True
182,Verkehrs & Handels Aktiengesellschaft,United Continental Corporation,True,False,False,False,False,False,True,True


## Search for AG

In [301]:
kw = 'tafel'
tfr500_hbs_moodys_tenebaum[tfr500_hbs_moodys_tenebaum['Master name'].str.contains(kw, case=False)]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenebaum
56,Deutsche Tafelglas Aktiengesellschaft (Detag).,NaN,False,False,True,False,False,True


## Comparison

### HB 32 and 34

In [302]:
hbs_merged = hb32_validated.merge(hb34_validated, on='Master name', suffixes=['_HB32', '_HB34'], how='outer')

In [303]:
hbs_merged['in HB 32'] = hbs_merged['firmname_HB32'].notna()
hbs_merged['in HB 34'] = hbs_merged['firmname_HB34'].notna()

In [304]:
priority_cols = ['Grossaktionär is a US comp', 'Director attached to a US company', 'US location linked to a director', 'US-sounding names', 'Other indicators are strong']
priority_map = {val: i for i, val in enumerate(priority_cols)}
# Function to handle priority + missing values
def choose_indicator(row):
    i1, i2 = row["indicator_HB32"], row["indicator_HB34"]
    if pd.isna(i1) and pd.isna(i2):
        return np.nan
    elif pd.isna(i1):
        return i2
    elif pd.isna(i2):
        return i1
    else:
        return i1 if priority_map[i1] <= priority_map[i2] else i2


# Pick whichever indicator has higher priority
hbs_merged["indicator_merged"] = hbs_merged.apply(choose_indicator, axis=1)

In [387]:
hbs_merged_tfr = hbs_merged.merge(tfr500, on='Master name', how='outer', indicator=True)
hbs_merged_tfr['in TFR'] = hbs_merged_tfr['_merge'].isin(['right_only', 'both'])
hbs_merged_tfr['in HBs'] = hbs_merged_tfr['_merge'].isin(['left_only', 'both'])
hbs_merged_tfr['in HB 32'] = hbs_merged_tfr['in HB 32'].fillna(False)
hbs_merged_tfr['in HB 34'] = hbs_merged_tfr['in HB 34'].fillna(False)

hbs_merged_tfr = hbs_merged_tfr.drop(columns=['_merge'])

/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/1461424142.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hbs_merged_tfr['in HB 32'] = hbs_merged_tfr['in HB 32'].fillna(False)
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/1461424142.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hbs_merged_tfr['in HB 34'] = hbs_merged_tfr['in HB 34'].fillna(False)


In [388]:
indicator_counts_HB32 = hbs_merged_tfr['indicator_HB32'].value_counts().rename("HB 32")
indicator_counts_HB34 = hbs_merged_tfr['indicator_HB34'].value_counts().rename("HB 34")
indicator_counts_merged = hbs_merged_tfr['indicator_merged'].value_counts().rename("merged")

indicator_summary = pd.concat([indicator_counts_HB32, indicator_counts_HB34, indicator_counts_merged], axis=1).fillna(0).astype(int)
indicator_summary = indicator_summary.reindex(priority_cols, fill_value=0)

# Add total row[]
indicator_summary.loc["\emph{Total}"] = indicator_summary.sum()
indicator_summary

<>:9: SyntaxWarning: invalid escape sequence '\e'
<>:9: SyntaxWarning: invalid escape sequence '\e'
/var/folders/44/38ww_5ts5z53mxh7j8kt4dt40000gn/T/ipykernel_90940/2859628371.py:9: SyntaxWarning: invalid escape sequence '\e'
  indicator_summary.loc["\emph{Total}"] = indicator_summary.sum()


,HB 32,HB 34,merged
Grossaktionär is a US comp,20,15,23
Director attached to a US company,7,9,12
US location linked to a director,51,42,56
US-sounding names,14,11,20
Other indicators are strong,3,7,8
\emph{Total},95,84,119


In [389]:
indicator_summary.to_latex('HB_AG_indicator_summary.tex', escape=False)

In [390]:
crosstab_HB_TFR = pd.crosstab(["indicator_merged"], hbs_merged_tfr["in TFR"]).reindex(priority_cols, fill_value=0)
crosstab_HB_TFR = crosstab_HB_TFR.rename(columns={True: "in TFR", False: "not in TFR"})
crosstab_HB_TFR.loc["Total"] = crosstab_HB_TFR.sum()

# Ensure flat columns (convert to str if needed)
crosstab_HB_TFR.columns = crosstab_HB_TFR.columns.map(str)

crosstab_HB_TFR = crosstab_HB_TFR.reset_index()
crosstab_HB_TFR = crosstab_HB_TFR.rename(columns={'indicator_merged': 'indicator'})

In [391]:
crosstab_HB_TFR.to_latex('HB_TFR_summary.tex', index=False)

In [368]:
# Replace empty strings with np.nan
hbs_merged_tfr['Master US firm name_HB32'] = hbs_merged_tfr['Master US firm name_HB32'].replace("", np.nan)
hbs_merged_tfr['Master US firm name_HB34'] = hbs_merged_tfr['Master US firm name_HB34'].replace("", np.nan)

hbs_parent_names = hbs_merged_tfr[hbs_merged_tfr['in HBs']==True]
hbs_parent_names = hbs_parent_names[['in HB 32', 'in HB 34', 'in TFR', 'Master US firm name_HB32', 'Master US firm name_HB34']]


# Function to merge names
def merge_name(row):
    n1, n2 = row['Master US firm name_HB32'], row['Master US firm name_HB34']
    
    # Check for empty strings or None
    n1_valid = n1 if pd.notna(n1) and n1.strip() != "" else None
    n2_valid = n2 if pd.notna(n2) and n2.strip() != "" else None
    
    if n1_valid:
        return n1_valid
    elif n2_valid:
        return n2_valid
    else:
        return np.nan

# Apply function
hbs_parent_names["US firm name_merged"] = hbs_parent_names.apply(merge_name, axis=1)

parent_summary = pd.DataFrame({
    "List": ["HB 32", "HB 34", "merged"],
    "Mentions parent name": [
        hbs_parent_names.loc[hbs_parent_names["in HB 32"] == 1, "Master US firm name_HB32"].notna().sum(),
        hbs_parent_names.loc[hbs_parent_names["in HB 34"] == 1, "Master US firm name_HB34"].notna().sum(),
        hbs_parent_names["US firm name_merged"].notna().sum()
    ],
    "Total": [
        hbs_parent_names["in HB 32"].sum(),
        hbs_parent_names["in HB 34"].sum(),
        len(hbs_parent_names)
    ]
})

parent_summary = pd.DataFrame({
    "HB 32": {
        "Mentions parent name": hbs_parent_names.loc[hbs_parent_names["in HB 32"] == 1, "Master US firm name_HB32"].notna().sum(),
        "Total": hbs_parent_names["in HB 32"].sum()
    },
    "HB 34": {
        "Mentions parent name": hbs_parent_names.loc[hbs_parent_names["in HB 34"] == 1, "Master US firm name_HB34"].notna().sum(),
        "Total": hbs_parent_names["in HB 34"].sum()
    },
    "merged": {
        "Mentions parent name": hbs_parent_names["US firm name_merged"].notna().sum(),
        "Total": len(hbs_parent_names)
    }
})

parent_summary.to_latex('HB_parent_mention_summary.tex')

parent_summary


,HB 32,HB 34,merged
Mentions parent name,41,30,48
Total,95,84,119


In [370]:
hbs_parent_names

,in HB 32,in HB 34,in TFR,Master US firm name_HB32,Master US firm name_HB34,US firm name_merged
0,False,True,True,NaN,NaN,NaN
3,True,True,False,NaN,NaN,NaN
4,True,True,True,General Motors Corporation (Overseas Operations),General Motors Corporation (Overseas Operations),General Motors Corporation (Overseas Operations)
5,True,True,False,NaN,NaN,NaN
7,True,True,False,NaN,NaN,NaN
...,...,...,...,...,...,...
171,True,False,False,NaN,NaN,NaN
172,False,True,False,NaN,NaN,NaN
173,True,True,False,Wm. Wrigley Jr. Company,Wm. Wrigley Jr. Company,Wm. Wrigley Jr. Company
174,False,True,False,NaN,Guardian Life Insurance Co. of America,Guardian Life Insurance Co. of America


In [380]:
hbs_parent_names['Mentions parent name'] = hbs_parent_names['US firm name_merged'].notna()
parent_summary_by_tfr = pd.crosstab(hbs_parent_names["Mentions parent name"], hbs_parent_names["in TFR"])
parent_summary_by_tfr = parent_summary_by_tfr.rename(columns={True: "in TFR", False: "not in TFR"})
parent_summary_by_tfr.index = parent_summary_by_tfr.index.map({True: "mentions parent name", False: "no parent name"})
parent_summary_by_tfr = parent_summary_by_tfr.reset_index()

parent_summary_by_tfr.to_latex('HB_parent_summary_by_tfr.tex', index=False)

In [ ]:
parent_summary = pd.DataFrame({
    "HB 32": {
        "Mentions parent name": hbs_parent_names.loc[hbs_parent_names["in HB 32"] == 1, "Master US firm name_HB32"].notna().sum(),
        "Total": hbs_parent_names["in HB 32"].sum()
    },
    "HB 34": {
        "Mentions parent name": hbs_parent_names.loc[hbs_parent_names["in HB 34"] == 1, "Master US firm name_HB34"].notna().sum(),
        "Total": hbs_parent_names["in HB 34"].sum()
    },
    "merged": {
        "Mentions parent name": hbs_parent_names["US firm name_merged"].notna().sum(),
        "Total": len(hbs_parent_names)
    }
})

,in HB 32,in HB 34,Master US firm name_HB32,Master US firm name_HB34,US firm name_merged
0,False,True,NaN,NaN,NaN
1,True,True,NaN,NaN,NaN
2,True,True,General Motors Corporation (Overseas Operations),General Motors Corporation (Overseas Operations),General Motors Corporation (Overseas Operations)
3,True,True,NaN,NaN,NaN
4,True,True,NaN,NaN,NaN
...,...,...,...,...,...
114,True,False,NaN,NaN,NaN
115,False,True,NaN,NaN,NaN
116,True,True,Wm. Wrigley Jr. Company,Wm. Wrigley Jr. Company,Wm. Wrigley Jr. Company
117,False,True,NaN,Guardian Life Insurance Co. of America,Guardian Life Insurance Co. of America


In [400]:
print(f"in HB 32 and in TFR: {hbs_merged_tfr[hbs_merged_tfr['in HB 32']& hbs_merged_tfr['in TFR']].shape[0]}")
print(f"in HB 34 and in TFR: {hbs_merged_tfr[hbs_merged_tfr['in HB 34']& hbs_merged_tfr['in TFR']].shape[0]}")

in HB 32 and in TFR: 29
in HB 34 and in TFR: 31


### Moody's + HBs

In [411]:
master_hbs_searched

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,True,True
3,A.G. vormals Seidel and Neumann,Corn Products Refining company,True,False,False,False,False,False,True,True
4,AG für Anlagewerte,"New Jersey Industries, Inc.",True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...
186,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False,True,True
187,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
188,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False,True,True
189,“Rustica” AG für Grunderwerk,United Continental Corporation,True,False,False,False,False,False,True,True


In [410]:
print('in HB 32 and in TFR: ' + str(master_hbs_searched[(master_hbs_searched['HB 32 validated']) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HB 32 and in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['HB 32 validated']) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in HB 34 and in TFR: ' + str(master_hbs_searched[(master_hbs_searched['HB 34 validated']) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HB 34 and in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['HB 34 validated']) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in HBs merged and in TFR: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated'])) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HBs merged and in Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated'])) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
master_hbs_searched[['TFR-500', 'Tenenbaum']].sum()

in HB 32 and in TFR: 29
in HB 32 and in Tenenbaum: 26
in HB 34 and in TFR: 31
in HB 34 and in Tenenbaum: 27
in HBs merged and in TFR: 37
in HBs merged and in Tenenbaum: 31


TFR-500      95
Tenenbaum    50
dtype: int64

In [418]:
print('in HB 32: ' + str(master_hbs_searched[(master_hbs_searched['HB 32 validated'])].shape[0]))
print('in HB 32 and in TFR: ' + str(master_hbs_searched[(master_hbs_searched['HB 32 validated']) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HB 32 and in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['HB 32 validated']) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in HB 32 and in TFR or Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['HB 32 validated']) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))
print('-------------------')
print('in HB 34: ' + str(master_hbs_searched[(master_hbs_searched['HB 34 validated'])].shape[0]))
print('in HB 34 and in TFR: ' + str(master_hbs_searched[(master_hbs_searched['HB 34 validated']) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HB 34 and in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['HB 34 validated']) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in HB 32 and in TFR or Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['HB 34 validated']) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))
print('-------------------')
print('in HB merged: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated']))].shape[0]))
print('in HB merged and in TFR: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated'])) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HB merged and in Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated'])) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in HB merged and in TFR or Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated'])) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))

in HB 32: 95
in HB 32 and in TFR: 29
in HB 32 and in Tenenbaum: 26
in HB 32 and in TFR or Tenenbaum: 37
-------------------
in HB 34: 84
in HB 34 and in TFR: 31
in HB 34 and in Tenenbaum: 27
in HB 32 and in TFR or Tenenbaum: 40
-------------------
in HB merged: 119
in HB merged and in TFR: 37
in HB merged and in Tenenbaum: 31
in HB merged and in TFR or Tenenbaum: 46


In [417]:
print('in Moodys 32: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 32'])].shape[0]))
print('in Moodys 32 and in TFR: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 32']) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in Moodys 32 and in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 32']) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in Moodys 32 and in TFR or Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 32']) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))
print('-------------------')
print('in Moodys 34: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 34'])].shape[0]))
print('in Moodys 34 and in TFR: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 34']) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in Moodys 34 and in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 34']) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in Moodys 32 and in TFR or Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['Moodys 34']) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))
print('-------------------')
print('in Moodys merged: ' + str(master_hbs_searched[((master_hbs_searched['Moodys 32']) | 
                    (master_hbs_searched['Moodys 34']))].shape[0]))
print('in Moodys merged and in TFR: ' + str(master_hbs_searched[((master_hbs_searched['Moodys 32']) | 
                    (master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in Moodys merged and in Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['Moodys 32']) | 
                    (master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in Moodys merged and in TFR or Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['Moodys 32']) | 
                    (master_hbs_searched['Moodys 34'])) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))

in Moodys 32: 24
in Moodys 32 and in TFR: 16
in Moodys 32 and in Tenenbaum: 13
in Moodys 32 and in TFR or Tenenbaum: 19
-------------------
in Moodys 34: 23
in Moodys 34 and in TFR: 15
in Moodys 34 and in Tenenbaum: 14
in Moodys 32 and in TFR or Tenenbaum: 19
-------------------
in Moodys merged: 30
in Moodys merged and in TFR: 19
in Moodys merged and in Tenenbaum: 17
in Moodys merged and in TFR or Tenenbaum: 24


In [420]:
print('in HB Moodys merged and in TFR: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated']) | 
                    (master_hbs_searched['Moodys 32']) |
                    (master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['TFR-500'])].shape[0]))
print('in HB Moodys merged and in Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated']) | 
                    (master_hbs_searched['Moodys 32']) |
                    (master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['Tenenbaum'])].shape[0]))
print('in HB Moodys merged and in TFR or Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated']) | 
                    (master_hbs_searched['Moodys 32']) |
                    (master_hbs_searched['Moodys 34'])) & 
                    ((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))

in HB Moodys merged and in TFR: 42
in HB Moodys merged and in Tenenbaum: 34
in HB Moodys merged and in TFR or Tenenbaum: 52


In [422]:
print('in TFR: ' + str(master_hbs_searched[(master_hbs_searched['TFR-500'])].shape[0]))
print('in Tenenbaum: ' + str(master_hbs_searched[(master_hbs_searched['Tenenbaum'])].shape[0]))
print('in TFR or Tenenbaum: ' + str(master_hbs_searched[((master_hbs_searched['TFR-500']) | (master_hbs_searched['Tenenbaum']))].shape[0]))

in TFR: 95
in Tenenbaum: 50
in TFR or Tenenbaum: 114


In [443]:
print('in TFR, and present in HB 32: ' + str(master_hbs_searched[(master_hbs_searched['TFR-500']) &
                    (master_hbs_searched['Present in HB 32'])].shape[0]))
print('in Tenenbaum, and present in HB 32: ' + str(master_hbs_searched[(master_hbs_searched['Tenenbaum']) &
                    (master_hbs_searched['Present in HB 32'])].shape[0]))
print('in TFR or Tenenbaum, and present in HB 32: ' + str(master_hbs_searched[((master_hbs_searched['TFR-500']) |
                    (master_hbs_searched['Tenenbaum'])) & 
                    (master_hbs_searched['Present in HB 32'])].shape[0]))
print('--------------')
print('in TFR, and present in HB 34: ' + str(master_hbs_searched[(master_hbs_searched['TFR-500']) &
                    (master_hbs_searched['Present in HB 34'])].shape[0]))
print('in Tenenbaum, and present in HB 34: ' + str(master_hbs_searched[(master_hbs_searched['Tenenbaum']) &
                    (master_hbs_searched['Present in HB 34'])].shape[0]))
print('in TFR or Tenenbaum, and present in HB 34: ' + str(master_hbs_searched[((master_hbs_searched['TFR-500']) |
                    (master_hbs_searched['Tenenbaum'])) & 
                    (master_hbs_searched['Present in HB 34'])].shape[0]))
print('--------------')
print('in TFR, and present in HB 32 or 34: ' + str(master_hbs_searched[(master_hbs_searched['TFR-500']) &
                    ((master_hbs_searched['Present in HB 32']) |
                     (master_hbs_searched['Present in HB 34']))].shape[0]))
print('in Tenenbaum, and present in HB 32 or 34: ' + str(master_hbs_searched[(master_hbs_searched['Tenenbaum']) &
                    ((master_hbs_searched['Present in HB 32']) |
                     (master_hbs_searched['Present in HB 34']))].shape[0]))
print('in TFR or Tenenbaum, and present in HB 32 or 34: ' + str(master_hbs_searched[((master_hbs_searched['TFR-500']) |
                    (master_hbs_searched['Tenenbaum'])) & 
                    ((master_hbs_searched['Present in HB 32']) |
                     (master_hbs_searched['Present in HB 34']))].shape[0]))

in TFR, and present in HB 32: 82
in Tenenbaum, and present in HB 32: 45
in TFR or Tenenbaum, and present in HB 32: 98
--------------
in TFR, and present in HB 34: 79
in Tenenbaum, and present in HB 34: 43
in TFR or Tenenbaum, and present in HB 34: 93
--------------
in TFR, and present in HB 32 or 34: 86
in Tenenbaum, and present in HB 32 or 34: 46
in TFR or Tenenbaum, and present in HB 32 or 34: 102


In [436]:
print('not identified, in TFR, and present in HB 32: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['TFR-500']) &
                    (master_hbs_searched['Present in HB 32'])].shape[0]))

print('not identified, in Tenenbaum, and present in HB 32: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['Tenenbaum']) &
                    (master_hbs_searched['Present in HB 32'])].shape[0]))

print('not identified, in TFR or Tenenbaum, and present in HB 32: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    ((master_hbs_searched['TFR-500']) |
                    (master_hbs_searched['Tenenbaum'])) &
                    (master_hbs_searched['Present in HB 32'])].shape[0]))
print('---------------')
print('not identified, in TFR, and present in HB 34: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['TFR-500']) &
                    (master_hbs_searched['Present in HB 34'])].shape[0]))

print('not identified, in Tenenbaum, and present in HB 34: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['Tenenbaum']) &
                    (master_hbs_searched['Present in HB 34'])].shape[0]))

print('not identified, in TFR or Tenenbaum, and present in HB 34: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    ((master_hbs_searched['TFR-500']) |
                    (master_hbs_searched['Tenenbaum'])) &
                    (master_hbs_searched['Present in HB 34'])].shape[0]))
print('---------------')
print('not identified, in TFR, and present in HB 32 or 34: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['TFR-500']) &
                    ((master_hbs_searched['Present in HB 32']) | 
                    (master_hbs_searched['Present in HB 34']))].shape[0]))

print('not identified, in Tenenbaum, and present in HB 32 or 34: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    (master_hbs_searched['Tenenbaum']) &
                    ((master_hbs_searched['Present in HB 32']) | 
                    (master_hbs_searched['Present in HB 34']))].shape[0]))

print('not identified, in TFR or Tenenbaum, and present in HB 32 or 34: ' + str(master_hbs_searched[((~master_hbs_searched['HB 32 validated']) & 
                    (~master_hbs_searched['HB 34 validated']) &
                    (~master_hbs_searched['Moodys 32']) &
                    (~master_hbs_searched['Moodys 34'])) & 
                    ((master_hbs_searched['TFR-500']) |
                    (master_hbs_searched['Tenenbaum'])) &
                    ((master_hbs_searched['Present in HB 32']) | 
                    (master_hbs_searched['Present in HB 34']))].shape[0]))

not identified, in TFR, and present in HB 32: 43
not identified, in Tenenbaum, and present in HB 32: 12
not identified, in TFR or Tenenbaum, and present in HB 32: 49
---------------
not identified, in TFR, and present in HB 34: 38
not identified, in Tenenbaum, and present in HB 34: 9
not identified, in TFR or Tenenbaum, and present in HB 34: 42
---------------
not identified, in TFR, and present in HB 32 or 34: 45
not identified, in Tenenbaum, and present in HB 32 or 34: 12
not identified, in TFR or Tenenbaum, and present in HB 32 or 34: 51


In [437]:
master_hbs_searched[(~master_hbs_searched['TFR-500']) & (master_hbs_searched['Tenenbaum'])]

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
0,"""Debag"" Betriebstoff A.G.",NaN,False,False,False,False,False,True,True,True
1,"""Dom"" Finanz A. G. für chemische Unternehmenungen",NaN,False,False,False,False,False,True,True,True
13,Allgemeine Elektricitäts-Gesellschaft,NaN,False,True,True,True,False,True,True,True
14,Allgemeine Transportmittel Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
22,Belton A.G.,NaN,False,False,False,False,False,True,False,False
37,Continentale Gummiwerke A.G.,NaN,False,False,False,False,False,True,True,False
39,Credit Aktiengesellschaft für Ford-Fahrzeuge.,NaN,False,True,True,False,False,True,True,True
51,Deutsche Johnson A. G.,NaN,False,False,False,False,False,True,True,False
53,Deutsche Libbey-Owens Gesellschaft für machine...,NaN,False,False,False,True,True,True,True,True
56,Deutsche Tafelglas Aktiengesellschaft (Detag).,NaN,False,False,True,False,False,True,True,True


In [ ]:
identified_AG = master_hbs_searched[(master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated']) | 
                    (master_hbs_searched['Moodys 32']) |
                    (master_hbs_searched['Moodys 34'])]

identified_AG #[['TFR-500', 'Tenenbaum']].sum()

,Master name,US parent,TFR-500,HB 32 validated,HB 34 validated,Moodys 32,Moodys 34,Tenenbaum,Present in HB 32,Present in HB 34
2,A. S. Hinds AG,Lehn & Pink Products Corporation,True,False,True,False,False,False,True,True
5,Actien-Bierbrauerei,NaN,False,True,True,False,False,False,True,True
6,Adam Opel Aktiengesellschaft,General Motors Corporation,True,True,True,True,True,True,True,True
7,Akt.-Ges. für Erwerb und Verwaltung von Grundb...,NaN,False,True,True,False,False,False,True,True
9,Aktiengesellschaft für Geschäftshausbau,NaN,False,False,False,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...
185,Warsteiner und Herzoglich Schleswig-Holsteinis...,NaN,False,True,False,False,False,False,True,True
186,Werschen-Weißenfelser Braunkohlen-Aktien-Gesel...,NaN,False,False,True,False,False,False,True,True
187,Wrigley Aktiengesellschaft,NaN,False,True,True,False,True,True,True,True
188,Zukunft Lebensversicherungsbank Aktiengesellsc...,NaN,False,False,True,False,False,False,True,True


In [382]:
identified_AG[['TFR-500', 'Tenenbaum']].sum()

TFR-500      42
Tenenbaum    34
dtype: int64

In [ ]:
master_hbs_searched[(master_hbs_searched['HB 32 validated']) | 
                    (master_hbs_searched['HB 34 validated']) | 
                    (master_hbs_searched['Moodys 32']) |
                    (master_hbs_searched['Moodys 34'])]